In [1]:
import tqdm
import requests
import pandas as pd
import geopandas as gpd

/opt/homebrew/lib/python3.10/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/l3/rl43cxg95q9dgp35dpn4hlgw0000gn/T/ipykernel_48355/4280674922.py:4: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. Ho

In [2]:
# Read Data
data = pd.read_excel("tickets_export.xlsx")
data

,Nummer,Erstellt,Lösungszeit,Meldeberechtigter,Organisation,Anfrager,Kanal,Lieferant,Serviceangebot,Aktualisiert,...,Unterkategorie,Status,Dringlichkeit,Priorität,Abteilung,Ort,Auswirkung,Gelöst,SLA def processing breached,SLA def reaction breached
0,INC475189,44773.541250,1162838.0,"Widmer Manus, JUSTICE-STAW-BJS-Biel",JUSTICE,NaN,Service Portal,NaN,SOF: Support Applikation Tribuna Anonymisierun...,44787.000023,...,Software,Geschlossen,Mittel,Tief,Staatsanwaltschaft Berner Jura-Seeland,"Spitalstrasse 11, 2501 Biel-Bienne",mehrere Arbeitsplätze,44773.692975,False,False
1,INC475187,44773.432211,826745.0,"Saxer Vera, GSI-ZSHKK","Gesundheits-, Sozial- und Integrationsdirektion",NaN,Service Portal,NaN,SOF-CS: Profile,44783.001019,...,Benutzer Administration,Geschlossen,Tief,Tief,Zentrum für Sozial- und Heilpädagogik Köniz-Ke...,"Landorfstrasse 94, 3098 Köniz",einzelner Arbeitsplatz,44775.291354,False,False
2,INC475186,44773.226042,1967711.0,"Lecomte Fanny, SID-AJV-RGBI-B",Sicherheitsdirektion,"Tedde Tamara, SID-AJV-RGMO",Service Portal,NaN,SOF: Fachapplikation nicht definiert (SID-AJV),44796.000475,...,Benutzer Administration,Geschlossen,Tief,Tief,AJV Regionalgefängnis Moutier,"Rue du Château 30b, 2740 Moutier",einzelner Standort,44788.649132,False,False
3,INC475185,44772.958310,1089274.0,"Schwander Sarah, SID-AJV-RGBU-SL",Sicherheitsdirektion,NaN,Service Portal,NaN,SOF-CS: Profile,44785.565648,...,Software,Geschlossen,Tief,Mittel,AJV_RGBU Sicherheit / Loge,"Dunantstrasse 9, 3400 Burgdorf",mehrere Standorte,44781.378044,False,False
4,INC475184,44772.912060,871685.0,"Lecomte Fanny, SID-AJV-RGBI-B",Sicherheitsdirektion,NaN,Service Portal,NaN,SOF: Support Applikation GINA (SID),44783.001007,...,System Administration,Geschlossen,Tief,Tief,AJV_RGMO Betreuung,"Rue du Château 30b, 2740 Moutier",einzelner Standort,44775.332801,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63507,INC510847,44896.295995,4033.0,"Ranieri Christine, FIN-SV-GBPS-JBSL-SL",Finanzdirektion,NaN,Service Portal,NaN,SOF: Betrieb Applikation NESKO-Frontend (FIN-SV),44902.000081,...,Software,Geschlossen,Hoch,Hoch,SV-GBPS-JBSL - Region Seeland,"Bahnhofplatz 10, 2502 Biel-Bienne",mehrere Standorte,44896.342303,False,False
63508,INC510846,44896.295949,11.0,"Trachsel Fritz, SID-SVSA-VPZBO-FUPBO",Sicherheitsdirektion,NaN,Telefon,NaN,SOF-CS: Profile,44896.295949,...,Benutzer Administration,Geschlossen,Tief,Tief,SVSA Führerprüfungen BO,"Tempelstrasse 30, 3608 Thun",einzelner Arbeitsplatz,44896.295949,False,False
63509,INC510844,44896.295556,611683.0,"Winkelmann Melanie, FIN-SV-GBPS-JBSL-SL",Finanzdirektion,NaN,Telefon,NaN,SOF: Betrieb Applikation NESKO-Frontend (FIN-SV),44903.375220,...,Software,Geschlossen,Hoch,Tief,SV-GBPS-JBSL - Region Seeland,"Bahnhofplatz 10, 2502 Biel-Bienne",einzelner Arbeitsplatz,44896.342500,False,False
63510,INC510843,44896.295000,615327.0,"Gfeller Eveline, BVD-TBA-DLZ",Bau- und Verkehrsdirektion,"Reusser Johann, BVD-TBA-DLZ",Telefon,NaN,SOF: Betrieb Applikation NESKO-Frontend (FIN-SV),44903.416840,...,Software,Geschlossen,Hoch,Tief,TBA-DLZ,"Reiterstrasse 11, 3013 Bern",einzelner Arbeitsplatz,44896.406250,False,False


In [3]:
# See if we can use "PLZ" to get the coordinates

street_array = data["Ort"].str.split(", ").str[0]
plz_array = data["Ort"].str.split(", ").str[-1].str.split(" ").str[0]
location = pd.concat([street_array, plz_array], axis=1)
location.columns = ["Street", "PLZ"]

# show how many different streets we have per plz
location.groupby("PLZ").Street.nunique().sort_values(ascending=False)

PLZ
3011            33
3000            19
3800            17
3600            13
2501            13
                ..
3283             1
3295             1
3296             1
3297             1
Rathausplatz     1
Name: Street, Length: 307, dtype: int64

too many adresses in one plz, we need to get specific coordinates

In [4]:
# function to get the coordinates of a location

def get_coordinates(location):
    url = f"https://nominatim.openstreetmap.org/search?q={location}&format=json"
    response = requests.get(url)
    data = response.json()
    return (data[0]['lon'], data[0]['lat']) if len(data) > 0 else (None, None)

In [5]:
# Usage example
location = "Spitalstrasse 11, 2501 Biel-Bienne"
coordinates = get_coordinates(location)

# geopandas set marker on specific coordinates
geometry = gpd.points_from_xy([coordinates[0]], [coordinates[1]])
gdf = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:4326")

# plot the dataframe as a map 
gdf.explore(min_zoom=18, max_zoom=18)

In [6]:
# get all the unique locations and their coordinates
unique_locations = data["Ort"].unique()
unique_coordinates = [get_coordinates(location) for location in tqdm.tqdm(unique_locations)]

# create a dataframe with the locations and coordinates
streetmapping = pd.DataFrame({"location": unique_locations, "coordinates": unique_coordinates})
streetmapping.head()

# save it 
streetmapping.to_excel("streetmapping.xlsx")

100%|██████████| 695/695 [06:09<00:00,  1.88it/s]


In [7]:
# get streets with no coordinates
missing_streets = streetmapping[streetmapping["coordinates"] == (None, None)].copy()

# replace \n and other stuff with a comma
missing_streets["location"] = missing_streets["location"].str.replace("\n", ", ").replace("\r", ", ").replace("\t", ", ").copy()

# remove middlepart of the location
# e.g. "Bernstrasse 5, Postfach 207, 3312 Fraubrunnen" -> "Bernstrasse 5, 3312 Fraubrunnen"
missing_streets["location"] = missing_streets["location"].str.split(", ").str[0] + ", " + missing_streets["location"].str.split(", ").str[-1].copy()

# get the coordinates for the missing streets
missing_coordinates = [get_coordinates(location) for location in tqdm.tqdm(missing_streets["location"])]

100%|██████████| 120/120 [01:03<00:00,  1.90it/s]


In [8]:
# add the coordinates to the dataframe
missing_streets["coordinates"] = missing_coordinates

# add the missing coordinates to the streetmapping dataframe
streetmapping.loc[streetmapping["coordinates"] == (None, None), "coordinates"] = missing_streets["coordinates"]

In [9]:
# get the missing streets once again
missing_streets = streetmapping[streetmapping["coordinates"] == (None, None)].copy()

# get last part of the location
# e.g. "Bernstrasse 5, Postfach 207, 3312 Fraubrunnen" -> "3312 Fraubrunnen"
missing_streets["location"] = missing_streets["location"].str.split(", ").str[-1:].str.join(", ").copy()

# get the coordinates for the missing streets
missing_coordinates = [get_coordinates(location) for location in tqdm.tqdm(missing_streets["location"])]

100%|██████████| 81/81 [00:44<00:00,  1.82it/s]


In [10]:
# add the coordinates to the dataframe
missing_streets["coordinates"] = missing_coordinates

# add the missing coordinates to the streetmapping dataframe
streetmapping.loc[streetmapping["coordinates"] == (None, None), "coordinates"] = missing_streets["coordinates"]

In [11]:
# get the missing streets once again
streetmapping[streetmapping["coordinates"] == (None, None)].copy()

,location,coordinates
306,"Dörfli 13c, 4917 Busswil b. Melchnau","(None, None)"
357,"Scheidgasse 2, 3703 Aeschi b. Spiez","(None, None)"
611,"Rue Fritz Marchand 51, 2615 Sonvillier","(None, None)"
656,"Viertel 130 E, 3815 Zweilüschtinen","(None, None)"


In [12]:
# convert coordinates to lon and lat columns
streetmapping["lon"] = streetmapping["coordinates"].str[0].astype(float)
streetmapping["lat"] = streetmapping["coordinates"].str[1].astype(float)

# remove the coordinates column
streetmapping = streetmapping.drop("coordinates", axis=1)

# save the dataframe
streetmapping.to_excel("streetmapping.xlsx")

In [13]:
# merge the dataframes on the location column
data = data.merge(streetmapping, left_on="Ort", right_on="location", how="left")

In [14]:
data.to_parquet("tickets_export_with_coords.parquet")

In [15]:
data_test = pd.read_parquet("tickets_export_with_coords.parquet")
data_test

,Nummer,Erstellt,Lösungszeit,Meldeberechtigter,Organisation,Anfrager,Kanal,Lieferant,Serviceangebot,Aktualisiert,...,Priorität,Abteilung,Ort,Auswirkung,Gelöst,SLA def processing breached,SLA def reaction breached,location,lon,lat
0,INC475189,44773.541250,1162838.0,"Widmer Manus, JUSTICE-STAW-BJS-Biel",JUSTICE,None,Service Portal,None,SOF: Support Applikation Tribuna Anonymisierun...,44787.000023,...,Tief,Staatsanwaltschaft Berner Jura-Seeland,"Spitalstrasse 11, 2501 Biel-Bienne",mehrere Arbeitsplätze,44773.692975,False,False,"Spitalstrasse 11, 2501 Biel-Bienne",7.243500,47.135292
1,INC475187,44773.432211,826745.0,"Saxer Vera, GSI-ZSHKK","Gesundheits-, Sozial- und Integrationsdirektion",None,Service Portal,None,SOF-CS: Profile,44783.001019,...,Tief,Zentrum für Sozial- und Heilpädagogik Köniz-Ke...,"Landorfstrasse 94, 3098 Köniz",einzelner Arbeitsplatz,44775.291354,False,False,"Landorfstrasse 94, 3098 Köniz",7.403668,46.920330
2,INC475186,44773.226042,1967711.0,"Lecomte Fanny, SID-AJV-RGBI-B",Sicherheitsdirektion,"Tedde Tamara, SID-AJV-RGMO",Service Portal,None,SOF: Fachapplikation nicht definiert (SID-AJV),44796.000475,...,Tief,AJV Regionalgefängnis Moutier,"Rue du Château 30b, 2740 Moutier",einzelner Standort,44788.649132,False,False,"Rue du Château 30b, 2740 Moutier",7.372722,47.280839
3,INC475185,44772.958310,1089274.0,"Schwander Sarah, SID-AJV-RGBU-SL",Sicherheitsdirektion,None,Service Portal,None,SOF-CS: Profile,44785.565648,...,Mittel,AJV_RGBU Sicherheit / Loge,"Dunantstrasse 9, 3400 Burgdorf",mehrere Standorte,44781.378044,False,False,"Dunantstrasse 9, 3400 Burgdorf",7.613708,47.065149
4,INC475184,44772.912060,871685.0,"Lecomte Fanny, SID-AJV-RGBI-B",Sicherheitsdirektion,None,Service Portal,None,SOF: Support Applikation GINA (SID),44783.001007,...,Tief,AJV_RGMO Betreuung,"Rue du Château 30b, 2740 Moutier",einzelner Standort,44775.332801,False,False,"Rue du Château 30b, 2740 Moutier",7.372722,47.280839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63507,INC510847,44896.295995,4033.0,"Ranieri Christine, FIN-SV-GBPS-JBSL-SL",Finanzdirektion,None,Service Portal,None,SOF: Betrieb Applikation NESKO-Frontend (FIN-SV),44902.000081,...,Hoch,SV-GBPS-JBSL - Region Seeland,"Bahnhofplatz 10, 2502 Biel-Bienne",mehrere Standorte,44896.342303,False,False,"Bahnhofplatz 10, 2502 Biel-Bienne",7.243172,47.133495
63508,INC510846,44896.295949,11.0,"Trachsel Fritz, SID-SVSA-VPZBO-FUPBO",Sicherheitsdirektion,None,Telefon,None,SOF-CS: Profile,44896.295949,...,Tief,SVSA Führerprüfungen BO,"Tempelstrasse 30, 3608 Thun",einzelner Arbeitsplatz,44896.295949,False,False,"Tempelstrasse 30, 3608 Thun",7.596349,46.747160
63509,INC510844,44896.295556,611683.0,"Winkelmann Melanie, FIN-SV-GBPS-JBSL-SL",Finanzdirektion,None,Telefon,None,SOF: Betrieb Applikation NESKO-Frontend (FIN-SV),44903.375220,...,Tief,SV-GBPS-JBSL - Region Seeland,"Bahnhofplatz 10, 2502 Biel-Bienne",einzelner Arbeitsplatz,44896.342500,False,False,"Bahnhofplatz 10, 2502 Biel-Bienne",7.243172,47.133495
63510,INC510843,44896.295000,615327.0,"Gfeller Eveline, BVD-TBA-DLZ",Bau- und Verkehrsdirektion,"Reusser Johann, BVD-TBA-DLZ",Telefon,None,SOF: Betrieb Applikation NESKO-Frontend (FIN-SV),44903.416840,...,Tief,TBA-DLZ,"Reiterstrasse 11, 3013 Bern",einzelner Arbeitsplatz,44896.406250,False,False,"Reiterstrasse 11, 3013 Bern",7.461530,46.953434
